### I did not intend to display the results; therefore, it says that errors occurred.

## 1. left medial Pulvinar

In [1]:
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

# List of subjects and prefix numbers
subjects = ['Linus', 'Curius', 'Linus1015', 'Pinocchio', 'Rio']  
prefix_numbers = ['002']  

# Iterate over each subject and prefix number
for subject in subjects:
    for prefix_number in prefix_numbers:
        
        # Load predrug CSV file
        predrug_file = f'netts/csv_files/long_correlation_matrix_{subject}_predrug_{prefix_number}.csv'
        
        if not os.path.exists(predrug_file):
            print(f"File for {subject} with prefix {prefix_number} not found. Skipping it...")
            continue
        
        df_predrug = pd.read_csv(predrug_file)
        
        # Dictionary to store mean values
        mean_values = {}
        
        for hemisphere in ['left', 'right']:
            prefix2 = 'CR_' if hemisphere == 'right' else 'CL_'
            
            # Filter DataFrame
            df_filtered = df_predrug[
                df_predrug['Variable1'].str.startswith('SL') &
                df_predrug['Variable1'].str.contains('MPul') &
                df_predrug['Variable2'].str.startswith(prefix2) &
                (df_predrug['Correlation'] != 1)
            ]
            
            # Calculate mean correlation
            mean_correlation = df_filtered['Correlation'].mean()
            mean_values[hemisphere] = mean_correlation
        
        # Print results
        #print(f"Mean correlations for {subject} with prefix {prefix_number} (only predrug):")
        #print(f"Left Hemisphere: {mean_values['left']:.4f}")
        #print(f"Right hemisphere: {mean_values['right']:.4f}")
        #print("="*50)


## 2. right medial Pul

In [2]:
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

# List of subjects and prefix numbers
subjects = ['Linus', 'Curius', 'Linus1015', 'Pinocchio', 'Rio']  
prefix_numbers = ['002']  

# Iterate over each subject and prefix number
for subject in subjects:
    for prefix_number in prefix_numbers:
        
        # Load predrug CSV file
        predrug_file = f'netts/csv_files/long_correlation_matrix_{subject}_predrug_{prefix_number}.csv'
        
        if not os.path.exists(predrug_file):
            print(f"File for {subject} with prefix {prefix_number} not found. Skipping it...")
            continue
        
        df_predrug = pd.read_csv(predrug_file)
        
        # Dictionary to store mean values
        mean_values = {}
        
        for hemisphere in ['left', 'right']:
            prefix2 = 'CR_' if hemisphere == 'right' else 'CL_'
            
            # Filter DataFrame
            df_filtered = df_predrug[
                df_predrug['Variable1'].str.startswith('SR') &
                df_predrug['Variable1'].str.contains('MPul') &
                df_predrug['Variable2'].str.startswith(prefix2) &
                (df_predrug['Correlation'] != 1)
            ]
            
            # Calculate mean correlation
            mean_correlation = df_filtered['Correlation'].mean()
            mean_values[hemisphere] = mean_correlation
        
        # Print results
        #print(f"Mean correlations for {subject} with prefix {prefix_number} (only predrug):")
        #print(f"Left hemisphere: {mean_values['left']:.4f}")
        #print(f"Right hemisphere: {mean_values['right']:.4f}")
        #print("="*50)


# Statistic and data sorting

In [3]:
import pandas as pd
import os
from scipy.stats import friedmanchisquare

# Subjekte und Präfixe
subjects = ['Linus', 'Curius', 'Linus1015', 'Pinocchio', 'Rio']
prefix_numbers = ['002']

# Ergebnisse sammeln
data = []

for subject in subjects:
    for prefix_number in prefix_numbers:
        file_path = f'netts/csv_files/long_correlation_matrix_{subject}_predrug_{prefix_number}.csv'

        if not os.path.exists(file_path):
            print(f"File not found: {file_path} — will be skipped.")
            continue

        df = pd.read_csv(file_path)
        entry = {'subject': subject}

        def safe_mean(label, df_filtered):
            if df_filtered.empty:
                print(f"Warning: No data for {subject} – {label}")
                return float('nan')
            mean_val = df_filtered['Correlation'].mean()
            if pd.isna(mean_val):
                print(f"Warning: Mean is NaN for {subject} – {label}")
            elif len(df_filtered) < 3:
                print(f"Attention: Only {len(df_filtered)} values for {subject} – {label}")
            return mean_val

        # Left Intra (SL → CL_)
        df_li = df[
            df['Variable1'].str.contains('SL') &
            df['Variable1'].str.contains('MPul') &
            df['Variable2'].str.startswith('CL_') &
            (df['Correlation'] != 1)
        ]
        entry['left_intra'] = safe_mean("left_intra (SL→CL_)", df_li)

        # Left Inter (SL → CR_)
        df_le = df[
            df['Variable1'].str.contains('SL') &
            df['Variable1'].str.contains('MPul') &
            df['Variable2'].str.startswith('CR_') &
            (df['Correlation'] != 1)
        ]
        entry['left_inter'] = safe_mean("left_inter (SL→CR_)", df_le)

        # Right Intra (SR → CR_)
        df_ri = df[
            df['Variable1'].str.contains('SR') &
            df['Variable1'].str.contains('MPul') &
            df['Variable2'].str.startswith('CR_') &
            (df['Correlation'] != 1)
        ]
        entry['right_intra'] = safe_mean("right_intra (SR→CR_)", df_ri)

        # Right Inter (SR → CL_)
        df_re = df[
            df['Variable1'].str.contains('SR') &
            df['Variable1'].str.contains('MPul') &
            df['Variable2'].str.startswith('CL_') &
            (df['Correlation'] != 1)
        ]
        entry['right_inter'] = safe_mean("right_inter (SR→CL_)", df_re)

        data.append(entry)

# Convert into dataframe
df_corr = pd.DataFrame(data)
print("\n=== Mittelwerte pro Subjekt ===")
print(df_corr)

df_corr_clean = df_corr.dropna(subset=['left_intra', 'left_inter', 'right_intra', 'right_inter'])
if df_corr_clean.shape[0] < len(subjects):
    #print(f"\n  {len(subjects) - df_corr_clean.shape[0]} Subjects were excluded due to missing values.")

if df_corr_clean.shape[0] >= 3:
    print("\n=== Friedman-Test: All 4 conditions ===")
    stat, p = friedmanchisquare(
        df_corr_clean['left_intra'],
        df_corr_clean['left_inter'],
        df_corr_clean['right_intra'],
        df_corr_clean['right_inter']
    )
    #print(f"Test statistic = {stat:.4f}, p-Wert = {p:.4f}")
else:
    #print("\n Too few valid subjects for the Friedman test (at least 3 subjects with complete data are required).")

    # Wilcoxon signed-rank tests for the desired pairs
    print("\n=== Wilcoxon signed-rank test: pairwise comparisons ===")
    
    # Left Intra vs Left Inter
    data_li = df_corr_clean['left_intra']
    data_le = df_corr_clean['left_inter']
    
    if len(data_li) > 1 and len(data_le) > 1:
        stat, p = wilcoxon(data_li, data_le)
        #print(f"left_intra vs left_inter: Teststatistik = {stat:.4f}, p-Wert = {p:.4f}")
    else:
        #print(f"left_intra vs left_inter: Too little data for the Wilcoxon test")

    # Right Intra vs Right Inter
    data_ri = df_corr_clean['right_intra']
    data_re = df_corr_clean['right_inter']
    
    if len(data_ri) > 1 and len(data_re) > 1:
        stat, p = wilcoxon(data_ri, data_re)
        #print(f"right_intra vs right_inter: Teststatistik = {stat:.4f}, p-Wert = {p:.4f}")
    else:
        #print(f"right_intra vs right_inter: Too little data for the Wilcoxon test")

else:
   # print("\n Too few valid subjects for the Friedman test (at least 3 subjects with complete data are required).")


IndentationError: expected an indented block after 'if' statement on line 78 (1246065783.py, line 81)

# Generating Plots

In [4]:
import matplotlib.pyplot as plt
import numpy as np

# Data for Left
subjects = df_corr_clean['subject'].values
left_intra = df_corr_clean['left_intra'].values
left_inter = df_corr_clean['left_inter'].values

# Data for Right
right_intra = df_corr_clean['right_intra'].values
right_inter = df_corr_clean['right_inter'].values

fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharey=True)

x_pos = [0, 1]  

# Left Plot
for i in range(len(subjects)):
    label = "Individual results" if i == 0 else None
    axs[0].plot(x_pos, [left_intra[i], left_inter[i]], marker='o', color='gray', alpha=0.6, label=label)

# Means (Left) - Rot
mean_left_intra = np.nanmean(left_intra)
mean_left_inter = np.nanmean(left_inter)
axs[0].plot(x_pos, [mean_left_intra, mean_left_inter], color='red', marker='D', markersize=6, linewidth=2, label='Mean')
axs[0].legend(loc='lower left', bbox_to_anchor=(0.0, 0.0), frameon=True)

axs[0].set_title('Medial pulvinar, left hemisphere')
axs[0].set_xticks(x_pos)
axs[0].set_xticklabels(['Intra', 'Inter'])
axs[0].set_ylabel('Mean correlation')
axs[0].grid(True)

# Right Plot
for i in range(len(subjects)):
    label = "Individual results" if i == 0 else None
    axs[1].plot(x_pos, [right_intra[i], right_inter[i]], marker='o', color='gray', alpha=0.6, label=label)
axs[1].set_title('Medial pulvinar , right hemisphere')
axs[1].set_xticks(x_pos)
axs[1].set_xticklabels(['Intra', 'Inter'])
axs[1].grid(True)

# Means (Right) - Rot
mean_right_intra = np.nanmean(right_intra)
mean_right_inter = np.nanmean(right_inter)
axs[1].plot(x_pos, [mean_right_intra, mean_right_inter], color='red', marker='D', markersize=6, linewidth=2, label='Mean')
axs[1].legend()

#plt.suptitle('Intra vs. Inter Correlation for each pulvinar')
plt.tight_layout()
#plt.savefig("medial_Pul_comparison_cortex.png", dpi=300)
#plt.show()


NameError: name 'df_corr_clean' is not defined